In [206]:
import datetime
from datetime import date
from datetime import datetime
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen
import glob
# import altair as alt
import time
import itertools
import numpy as np
import operator

pd.options.display.max_columns = 100


In [207]:
#Getting Player Info Files
# Sravan's paths
# player_info_england = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_data_df_england.csv')
# player_info_italy = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_data_df_italy.csv')
# player_info_spain = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_data_df_spain.csv')
# player_info_france = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_data_df_france.csv')
# player_info_germany = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_data_df_germany.csv')


# Renzo's paths
player_info_england = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_england.csv')
player_info_italy = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_italy.csv')
player_info_spain = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_spain.csv')
player_info_france = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_france.csv')
player_info_germany = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_germany.csv')


In [208]:
# Load TM Profile Datasets

df_1 = pd.read_csv('player_profile_df_1.csv')
df_2 = pd.read_csv('player_profile_df_2.csv')
df_3 = pd.read_csv('player_profile_df_3.csv')

tm_profile_df = pd.concat([df_1, df_2])
tm_profile_df = pd.concat([tm_profile_df, df_3])

In [209]:
player_inf_lst = [player_info_england, player_info_italy, player_info_spain, player_info_france, player_info_germany]
player_info_df = pd.concat(player_inf_lst)
player_info_df.shape

(35827, 15)

In [210]:
# Cleaning repeated players

player_info_df_nodups = player_info_df.drop_duplicates()
print(len(player_info_df))
print(len(player_info_df_nodups))

35827
10720


In [211]:
#Loading Player Injuries

# Sravan's Path
# player_injuries_df = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_injuries_df.csv')

# Renzo's Path
player_injuries_df = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_injuries_df.csv')

len(player_injuries_df)

55216

In [212]:
print(player_injuries_df.columns)
print(player_info_df_nodups.columns)

Index(['Season', 'Injury', 'from', 'until', 'Days', 'Games missed', 'TMId',
       'name'],
      dtype='object')
Index(['FBRefId', 'PlayerName', 'PlayerFullName', 'Nationality', 'Photo',
       'Birth', 'Height', 'Weight', 'Position', 'Foot', 'FootAbility',
       'InternationalReputation', 'PlayerUrl', 'Twitter', 'Instagram'],
      dtype='object')


In [213]:
# Load TMID to FBRefID reference table

# Sravan's Path
# fbref_to_tm_df = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/CSV files/fbref_to_tm_mapping.csv', encoding='latin-1')

# Renzo's Path
fbref_to_tm_df = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV files/fbref_to_tm_mapping.csv')

In [214]:
# Stripping FBRefID and TMID from other information and creating individual columns for them

# fbref_to_tm_df['FBRefID'] = fbref_to_tm_df['UrlFBref'].apply(lambda player: player[(player.find("players/") + len("players/")):(player.find("/matchlogs"))])

fbref_to_tm_df['FBRefID'] = fbref_to_tm_df['UrlFBref'].str.split('/').str[5]
fbref_to_tm_df['TMID'] = fbref_to_tm_df['UrlTmarkt'].str.split('/').str[6]

fbref_to_tm_df.head()


,PlayerFBref,UrlFBref,UrlTmarkt,TmPos,FBRefID,TMID
0,Aaron Connolly,https://fbref.com/en/players/27c01749/Aaron-Co...,https://www.transfermarkt.com/aaron-connolly/p...,Centre-Forward,27c01749,434207
1,Aaron Cresswell,https://fbref.com/en/players/4f974391/Aaron-Cr...,https://www.transfermarkt.com/aaron-cresswell/...,Left-Back,4f974391,92571
2,Aarón Escandell,https://fbref.com/en/players/67669ce7/Aaron-Es...,https://www.transfermarkt.com/aaron-escandell/...,Goalkeeper,67669ce7,284430
3,Aaron Herzog,https://fbref.com/en/players/565c3fe4/Aaron-He...,https://www.transfermarkt.com/aaron-herzog/pro...,Attacking Midfield,565c3fe4,276566
4,Aaron Hickey,https://fbref.com/en/players/1780bb4a/Aaron-Hi...,https://www.transfermarkt.com/aaron-hickey/pro...,Left-Back,1780bb4a,591949


In [215]:
# converting TMId to string in player_injuries_df

player_injuries_df['TMId'] = player_injuries_df['TMId'].astype(str)
# player_injuries_df.head()

In [216]:
# Merging on intersection of player_injuries_df and fbref_to_tm_df on columns TMId and TMID respectively

player_injuries_df_2 = pd.merge(left=player_injuries_df, right=fbref_to_tm_df, left_on='TMId', right_on='TMID', how='inner')

# player_injuries_df_2.head()

In [217]:
# Determining number of unique player in our data

len(player_injuries_df_2['name'].unique())

4614

In [218]:
# removing duplicates from player_info_df

player_info_df = player_info_df.drop_duplicates()

# player_info_df.head()

In [219]:
# Merging Player Injuries with FBRef Profiles

player_injuries_info_df = pd.merge(left=player_injuries_df_2, right=player_info_df, left_on='FBRefID', right_on='FBRefId', how='inner')

len(player_injuries_info_df)

32584

In [220]:
# finding number of unique players
 
len(player_injuries_info_df['name'].unique())

4586

In [221]:
# Merge with TM Profile information

tm_profile_df['TMId'] = tm_profile_df['TMId'].astype(str)
player_injuries_profile_final = pd.merge(left=player_injuries_info_df, right=tm_profile_df, left_on='TMId', right_on='TMId', how='inner')
player_injuries_profile_final.head()

,Season,Injury,from,until,Days,Games missed,TMId,name_x,PlayerFBref,UrlFBref,UrlTmarkt,TmPos,FBRefID,TMID,FBRefId,PlayerName,PlayerFullName,Nationality,Photo,Birth,Height,Weight,Position,Foot,FootAbility,InternationalReputation,PlayerUrl,Twitter,Instagram,Place of birth:,Citizenship:,Height:,Position:,Contract expires:,Former International:,Caps/Goals:,League level:,Joined:,Contract until:,name_y,Date of birth,Last club:,Most games for:,Unknown since:,Date of birth:,Age:,Foot:,Player agent:,Current club:,Agent:,National player:,Name in home country:,Outfitter:,Current international:,Social-Media:,Full name:,On loan from:,Contract there expires:,Contract option:,Retired since:,club:,Without Club since:,2nd club:,country:,Company:,Date of death:,Career break since:,3nd club:,Ban since:
0,18/19,Hamstring Injury,"Sep 29, 2018","Oct 27, 2018",28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16/17,Calf Strain,"Mar 18, 2017","Apr 17, 2017",30 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15/16,Calf Injury,"Mar 3, 2016","Mar 31, 2016",28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14/15,Neck Injury,"May 11, 2015","May 25, 2015",14 days,2,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13/14,Shoulder Injury,"Apr 24, 2014","Jul 17, 2014",84 days,4,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermark

In [222]:
# Creating new columns with features

player_injuries_profile_final = player_injuries_profile_final[player_injuries_profile_final['from'] != '-']
player_injuries_profile_final = player_injuries_profile_final[player_injuries_profile_final['until'] != '-']
player_injuries_profile_final['injury_year'] = player_injuries_profile_final['from'].apply(lambda x: datetime.strptime(x, '%b %d, %Y').year)
player_injuries_profile_final['injury_week'] = player_injuries_profile_final['from'].apply(lambda x: datetime.strptime(x, '%b %d, %Y').strftime('%V'))
player_injuries_profile_final['release_week'] = player_injuries_profile_final['until'].apply(lambda x: datetime.strptime(x, '%b %d, %Y').strftime('%V'))
player_injuries_profile_final['from'] = pd.to_datetime(player_injuries_profile_final['from'])
player_injuries_profile_final['until'] = pd.to_datetime(player_injuries_profile_final['until'])

In [223]:
# Exploding Dataframe to weekly basis

# player_injuries_profile_final['from']=player_injuries_profile_final.apply(lambda x : pd.date_range(start=x['from'], end=x['until'], freq='W') ,axis=1)
# player_injuries_profile_final = player_injuries_profile_final.explode('from')
# player_injuries_profile_final['until']=player_injuries_profile_final['from']+pd.tseries.offsets.MonthEnd()

# player_injuries_profile_final.head(50)

def ran(row):
    return list(pd.date_range(row['from'], row['until'], freq='w'))

player_injuries_profile_final['injury_week'] = player_injuries_profile_final['injury_week'].astype(int)
player_injuries_profile_final['release_week'] = player_injuries_profile_final['release_week'].astype(int)
player_injuries_profile_final['current_week'] = player_injuries_profile_final.apply(ran, axis = 1)

player_injuries_profile_final = player_injuries_profile_final.explode('current_week')




In [224]:
player_injuries_profile_final.head()

,Season,Injury,from,until,Days,Games missed,TMId,name_x,PlayerFBref,UrlFBref,UrlTmarkt,TmPos,FBRefID,TMID,FBRefId,PlayerName,PlayerFullName,Nationality,Photo,Birth,Height,Weight,Position,Foot,FootAbility,InternationalReputation,PlayerUrl,Twitter,Instagram,Place of birth:,Citizenship:,Height:,Position:,Contract expires:,Former International:,Caps/Goals:,League level:,Joined:,Contract until:,name_y,Date of birth,Last club:,Most games for:,Unknown since:,Date of birth:,Age:,Foot:,Player agent:,Current club:,Agent:,National player:,Name in home country:,Outfitter:,Current international:,Social-Media:,Full name:,On loan from:,Contract there expires:,Contract option:,Retired since:,club:,Without Club since:,2nd club:,country:,Company:,Date of death:,Career break since:,3nd club:,Ban since:,injury_year,injury_week,release_week,current_week
0,18/19,Hamstring Injury,2018-09-29,2018-10-27,28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,39,43,2018-09-30
0,18/19,Hamstring Injury,2018-09-29,2018-10-27,28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,39,43,2018-10-07
0,18/19,Hamstring Injury,2018-09-29,2018-10-27,28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,39,43,2018-10-14
0,18/19,Hamstring Injury,2018-09-29,2018-10-27,28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,39,43,2018-10-21
1,16/17,Calf Strain,2017-03-18,2017-04-

In [225]:
player_injuries_profile_final['current_week'] = player_injuries_profile_final['current_week'].apply(lambda x: x.week) #strftime('%W'))

In [230]:
player_injuries_profile_final[player_injuries_profile_final['current_week'].isna()]

,Season,Injury,from,until,Days,Games missed,TMId,name_x,PlayerFBref,UrlFBref,UrlTmarkt,TmPos,FBRefID,TMID,FBRefId,PlayerName,PlayerFullName,Nationality,Photo,Birth,Height,Weight,Position,Foot,FootAbility,InternationalReputation,PlayerUrl,Twitter,Instagram,Place of birth:,Citizenship:,Height:,Position:,Contract expires:,Former International:,Caps/Goals:,League level:,Joined:,Contract until:,name_y,Date of birth,Last club:,Most games for:,Unknown since:,Date of birth:,Age:,Foot:,Player agent:,Current club:,Agent:,National player:,Name in home country:,Outfitter:,Current international:,Social-Media:,Full name:,On loan from:,Contract there expires:,Contract option:,Retired since:,club:,Without Club since:,2nd club:,country:,Company:,Date of death:,Career break since:,3nd club:,Ban since:,injury_year,injury_week,release_week,current_week
5,12/13,Fractured Finger,2013-02-04,2013-02-07,3 days,-,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,6,6,NaN
12,07/08,Hip problems,2007-12-24,2007-12-24,0 days,-,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007,52,52,NaN
24,20/21,Knock,2020-11-27,2020-11-28,1 days,-,46741,David Luiz,David Luiz,https://fbref.com/en/players/c0e27d1a/David-Luiz,https://www.transfermarkt.com/david-luiz/profi...,Centre-Back,c0e27d1a,46741,c0e27d1a,David Luiz,David Luiz Moreira Marinho,Brazil,https://fbref.com/req/202005121/images/headsho...,"April 22, 1987",189.0,84.0,DF-MF (CB-DM),RIGHT,78.0,4x Domestic League Champion||2015-16 Coupe de ...,https://fbref.com/en/players/c0e27d1a/David-Luiz,https://twitter.com/DavidLuiz_4,https://instagram.com/davidluiz_4/,Diadema,Brazil Portugal,"1,89 m",Defender - Centre-Back,"Dec 31, 2022",Brazil,57/3,First Tier,"Sep 11, 2021","Dec 31, 2022",David Luiz,NaN,NaN,NaN,NaN,"Apr 22, 1987",34.0,right,Bertolucci Sports,Clube de Regatas do Flamengo,NaN,NaN,NaN,adidas,NaN,NaN,David Luiz Moreira Marinho,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,48,48,NaN
55,14/15,Hip Injury,2015-02-02,2015-02-05,3 days,1,88721,Jeffrey Bruma,Jeffrey Bruma,https://fbref.com/en/players/fbffc4b1/Jeffrey-...,https://www.transfermarkt.com/jeffrey-bruma/pr...,Centre-Back,fbffc4b1,88721,fbffc4b1,Jeffrey Bruma,Jeffrey Kevin van Homoet Bruma,Netherlands,https://fbref.com/req/202005121/images/headsho...,"November 13, 1991",189.0,86.0,DF (CB-FB),RIGHT,86.0,2x Dutch Eredivisie Champion,https://fbref.com/en/players/fbffc4b1/Jeffrey-...,NaN,NaN,Rotterdam,Netherlands Suriname,"1,89 m",Defender - Centre-Back,"Jun 30, 2023",Netherlands,25/1,First Tier,"Aug 4, 2021","Jun 30, 2023",Jeffrey Bruma,NaN,NaN,NaN,NaN,"Nov 13, 1991",30.0,right,Juzzt Football,Kasimpasa,NaN,NaN,Jeffrey Kevin van Homoet Bruma,NaN,NaN,NaN,NaN,NaN,NaN,Opt

In [232]:
player_injuries_profile_final['current_week'] = player_injuries_profile_final['current_week'].fillna(player_injuries_profile_final['injury_week'])

In [255]:
player_injuries_profile_final[player_injuries_profile_final['name_x'] == 'Manuel Neuer'][:50]

,Season,Injury,from,until,Days,Games missed,TMId,name_x,PlayerFBref,UrlFBref,UrlTmarkt,TmPos,FBRefID,TMID,FBRefId,PlayerName,PlayerFullName,Nationality,Photo,Birth,Height,Weight,Position,Foot,FootAbility,InternationalReputation,PlayerUrl,Twitter,Instagram,Place of birth:,Citizenship:,Height:,Position:,Contract expires:,Former International:,Caps/Goals:,League level:,Joined:,Contract until:,name_y,Date of birth,Last club:,Most games for:,Unknown since:,Date of birth:,Age:,Foot:,Player agent:,Current club:,Agent:,National player:,Name in home country:,Outfitter:,Current international:,Social-Media:,Full name:,On loan from:,Contract there expires:,Contract option:,Retired since:,club:,Without Club since:,2nd club:,country:,Company:,Date of death:,Career break since:,3nd club:,Ban since:,injury_year,injury_week,release_week,current_week
11150,21/22,Tear in a joint capsule,2021-08-30,2021-09-02,3 days,-,17259,Manuel Neuer,Manuel Neuer,https://fbref.com/en/players/8778c910/Manuel-N...,https://www.transfermarkt.com/manuel-neuer/pro...,Goalkeeper,8778c910,17259,8778c910,Manuel Neuer,Manuel Peter Neuer,Germany,https://fbref.com/req/202005121/images/headsho...,"March 27, 1986",193.0,92.0,GK,RIGHT,79.0,2020 The Best FIFA Men's Goalkeeper||8x Bundes...,https://fbref.com/en/players/8778c910/Manuel-N...,https://twitter.com/Manuel_Neuer,https://instagram.com/manuelneuer/,Gelsenkirchen,Germany,"1,93 m",Goalkeeper,"Jun 30, 2023",NaN,108/0,First Tier,"Jul 1, 2011","Jun 30, 2023",Manuel Neuer,NaN,NaN,NaN,NaN,"Mar 27, 1986",35.0,right,PRO Profil GmbH,Bayern Munich,PRO Profil GmbH,Germany,Manuel Peter Neuer,adidas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,35,35,35.0
11151,21/22,Tear in a joint capsule,2021-08-23,2021-08-25,2 days,1,17259,Manuel Neuer,Manuel Neuer,https://fbref.com/en/players/8778c910/Manuel-N...,https://www.transfermarkt.com/manuel-neuer/pro...,Goalkeeper,8778c910,17259,8778c910,Manuel Neuer,Manuel Peter Neuer,Germany,https://fbref.com/req/202005121/images/headsho...,"March 27, 1986",193.0,92.0,GK,RIGHT,79.0,2020 The Best FIFA Men's Goalkeeper||8x Bundes...,https://fbref.com/en/players/8778c910/Manuel-N...,https://twitter.com/Manuel_Neuer,https://instagram.com/manuelneuer/,Gelsenkirchen,Germany,"1,93 m",Goalkeeper,"Jun 30, 2023",NaN,108/0,First Tier,"Jul 1, 2011","Jun 30, 2023",Manuel Neuer,NaN,NaN,NaN,NaN,"Mar 27, 1986",35.0,right,PRO Profil GmbH,Bayern Munich,PRO Profil GmbH,Germany,Manuel Peter Neuer,adidas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,34,34,34.0
11152,18/19,Torn Muscle Fibre,2019-04-15,2019-05-19,34 days,6,17259,Manuel Neuer,Manuel Neuer,https://fbref.com/en/players/8778c910/Manuel-N...,https://www.transfermarkt.com/manuel-neuer/pro...,Goalkeeper,8778c910,17259,8778c910,Manuel Neuer,Manuel Peter Neuer,Germany,https://fbref.com/req/202005121/images/headsho...,"March 27, 1986",193.0,92.0,GK,RIGHT,79.0,2020 The Best FIFA Men's Goalkeeper||8x Bundes...,https://fbref.com/en/players/8778c910/Manuel-N...,https://twitter.com/Manuel_Neuer,https://instagram.com/manuelneuer/,Gelsenkirchen,Germany,"1,93 m",Goalkeeper,"Jun 30, 2023",NaN,108/0,First Tier,"Jul 1, 2011","Jun 30, 2023",Manuel Neuer,NaN,NaN,NaN,NaN,"Mar 27, 1986",35.0,right,PRO Profil GmbH,Bayern Munich,PRO Profil GmbH,Germany,Manuel Peter Neuer,adidas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,16,20,16.0
11152,18/19,Torn Muscle Fibre,2019-04-15,2019-05-19,34 days,6,17259,Manuel Neuer,Manuel Neuer,https://fbref.com/en/players/8778c910/Manuel-N...,https://www.transfermarkt.com/manuel-neuer/pro...,Goalkeeper,8778c910,17259,8778c910,Manuel Neuer,Manuel Peter Neuer,Germany,https://fbref.com/req/202005121/images/headsho...,"March 27, 1986",193.0,92.0,GK,RIGHT,79.0,2020 The Best FIFA Men's Goalkeeper||8x Bundes...,https://fbref.com/en/players/8778c910/Manuel-N...,https://twitter.com/Manuel_Neuer,https://instagram.com/manuelneuer/,Gelsenkirchen,Germany,"1,93 m",Goalkeeper,"Jun 30, 2023",NaN,10

In [33]:
# Loading Match Dataframes

# Renzo's Path
total_match_logs_df = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/consolidated_df.csv')

/Users/renzomaldonado/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (19,20,21,23,24,25,26,27,28,29,30,31,32,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [37]:
print(total_match_logs_df.columns)
print(player_injuries_profile_final.columns)

Index(['Date', 'Day', 'Comp', 'Round', 'Venue', 'Result', 'Squad', 'Opponent',
       'Start', 'Pos', 'Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY',
       'CrdR', 'Touches', 'Press', 'Tkl', 'Int', 'Blocks', 'xG', 'npxG', 'xA',
       'SCA', 'GCA', 'Cmp', 'Att', 'Cmp%', 'Prog', 'Carries', 'Prog.1', 'Succ',
       'Att.1', 'name', 'FBRefID', 'Fls', 'Fld', 'Off', 'Crs', 'TklW', 'OG',
       'PKwon', 'PKcon'],
      dtype='object')
Index(['Season', 'Injury', 'from', 'until', 'Days', 'Games missed', 'TMId',
       'name_x', 'PlayerFBref', 'UrlFBref', 'UrlTmarkt', 'TmPos', 'FBRefID',
       'TMID', 'FBRefId', 'PlayerName', 'PlayerFullName', 'Nationality',
       'Photo', 'Birth', 'Height', 'Weight', 'Position', 'Foot', 'FootAbility',
       'InternationalReputation', 'PlayerUrl', 'Twitter', 'Instagram',
       'Place of birth:', 'Citizenship:', 'Height:', 'Position:',
       'Contract expires:', 'Former International:', 'Caps/Goals:',
       'League level:', 'Joined:', 'Contract u

In [32]:
# Saving player_injuries_profile_final to Google Drive

# Renzo's path

player_injuries_profile_final.to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_injuries_profile_final.csv', index=False)